In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib.image import imread
import seaborn as sns

In [ ]:
data_dir = 'cell_images'

In [ ]:
os.listdir(data_dir)

In [ ]:
train_path = data_dir + '/train'
test_path = data_dir + '/test'

In [ ]:
os.listdir(test_path)

In [ ]:
os.listdir(train_path + '/parasitized')

In [ ]:
par_cell = train_path+ '/parasitized/' + 'C68P29N_ThinF_IMG_20150819_134504_cell_162.png'
plt.imshow(plt.imread(par_cell))

In [ ]:
os.listdir(train_path + '/uninfected')

In [ ]:
par_cell = train_path+ '/uninfected/' + 'C129P90ThinF_IMG_20151004_133703_cell_58.png'
plt.imshow(plt.imread(par_cell))

In [ ]:
imread(par_cell).shape

In [ ]:
dim1 = []
dim2 = []

for img in os.listdir(test_path + '/uninfected'):
    img_read = imread(test_path + '/uninfected/' + img)
    dim1.append(img_read.shape[0])
    dim2.append(img_read.shape[1])

sns.jointplot(dim1, dim2)

In [ ]:
img_shape = (130, 130, 3)

In [ ]:
image_gen = ImageDataGenerator(rotation_range = 20, 
                               width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               shear_range =  0.1,
                               zoom_range = 0.1,
                               fill_mode = 'nearest'
                              )

In [ ]:
par_img = plt.imread(par_cell)

plt.imshow(par_img)

In [ ]:
plt.imshow(image_gen.random_transform(par_img))

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=img_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=img_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=img_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
early_stop = EarlyStopping(patience = 2, monitor = 'val_loss')

In [ ]:
batch_size = 16

In [ ]:
train_img_gen = image_gen.flow_from_directory(directory = train_path+'/',
                                             target_size = img_shape[:2],
                                             color_mode = 'rgb',
                                             batch_size = batch_size,
                                             class_mode = 'binary')

In [ ]:
test_img_gen = image_gen.flow_from_directory(directory = test_path+'/',
                                                 target_size = img_shape[:2],
                                             color_mode = 'rgb',
                                             batch_size = batch_size,
                                             class_mode = 'binary',
                                            shuffle = False)

In [ ]:
train_img_gen.class_indices

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:

results = model.fit_generator(train_img_gen,epochs=20,
                              validation_data=train_img_gen,
                             callbacks=[early_stop])

In [ ]:
results.predict_generator(test_img_gen)